In [144]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box

In [145]:
coord_ref_sys = "EPSG:25829"

In [146]:
fire_data = gpd.read_file("data/fire_data/modis.ba.poly.shp")
fire_data.head()

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,SCLEROPH,TRANSIT,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,geometry
0,2,2016-08-28 00:00:00,2022-01-26 11:57:54.973474,AL,Korcë,Konispol,67,0,0,0,19.402985074597908,16.417910447736688,10.447761194014257,53.731343283501886,0,0,0,FireSeason,"POLYGON ((20.18901 39.74908, 20.18909 39.75006..."
1,888,2016-08-08 00:00:00,2022-01-26 11:57:54.973474,PT,Área Metropolitana do Porto,Covelo de Paivó e Janarde,26593,0.6020922706404734,0.2558892150222012,0.2220215247986746,0,56.893956498833234,38.661849928501404,3.2174305712350297,0.14675999096861542,0,9.843358251326316,FireSeason,"POLYGON ((-8.06251 40.84474, -8.06033 40.8436,..."
2,19,2016-08-11 00:00:00,2022-01-26 11:57:54.973474,PT,Tâmega e Sousa,Caçarilhe e Infesta,81,0,1.17647058823391,0,0,67.05882352933287,19.99999999997647,11.7647058823391,0,0,0,FireSeason,"POLYGON ((-8.05657 41.40623, -8.05945 41.4048,..."
3,87,2016-07-27 00:00:00,2022-01-26 11:57:54.973474,TR,??rnak,N.A.,72,0,0,0,0,70.83333333323495,22.222222222191355,0,6.9444444444348,0,0,FireSeason,"POLYGON ((42.50247 37.53773, 42.5029 37.53451,..."
4,368,2016-08-24 00:00:00,2022-01-26 11:57:54.973474,IT,Genova,Bogliasco,267,11.654135338341485,5.639097744358783,0.37593984962391885,0,7.894736842102295,74.06015037591202,0.37593984962391885,0,0,5.453094633641743,FireSeason,"POLYGON ((9.06157 44.39255, 9.06266 44.39417, ..."


In [147]:
# Natural Earth - public domain dataset of global geography
# Admin 0 – Countries dataset has country boundaries
# https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
world = gpd.read_file("data/country_boundaries/ne_110m_admin_0_countries.shp")
portugal = world[world["NAME"] == "Portugal"]

# mainland = portugal.loc[portugal.geometry.area.idxmax()]
# minx, miny, maxx, maxy = -9.526570603869715 36.83826854099627 -6.389087693700915 42.28046865495034

# https://epsg.io/25829
portugal_proj = portugal.to_crs(coord_ref_sys) # degrees -> meters
# mainland = portugal_proj.iloc[portugal_proj.geometry.area.argmax()]
mainland = portugal_proj.iloc[portugal_proj.geometry.area.argmax()]
mainland_polygon = mainland.geometry

minx, miny, maxx, maxy = mainland_polygon.bounds
print(minx, miny, maxx, maxy)

454235.1439060115 4077542.239783567 718319.3451133408 4681179.434351782


In [148]:
grid_size = 2000 # meters
# n_cols = int((maxx - minx) // grid_size)
# n_rows = int((maxy - miny) // grid_size)
# print(n_cols, n_rows)

# pt_width = maxx - minx
# pt_height = maxy - miny
# check_n_cols = pt_width / grid_size
# check_nrows = pt_height / grid_size
# print(check_n_cols, check_nrows)

# x_coords = minx + np.arange(n_cols) * grid_size
# y_coords = miny + np.arange(n_rows) * grid_size
x_coords = np.arange(minx, maxx, grid_size)
y_coords = np.arange(miny, maxy, grid_size)
# print(x_coords, y_coords)

In [152]:
# create grid
grid_squares = []
for x in x_coords:
    for y in y_coords:
        # this makes a full rectangle grid
        cell = box(x, y, x + grid_size, y + grid_size)
        # only add cells that are inside mainland portugal
        if mainland_polygon.contains(cell):
            grid_squares.append(cell)

grid = gpd.GeoDataFrame({"geometry": grid_squares}, crs=coord_ref_sys)
grid.head()
# each entry is lower-left, upper-left, upper-right, lower-right, lower-left points of each square

,geometry
0,"POLYGON ((458235.144 4285542.24, 458235.144 42..."
1,"POLYGON ((458235.144 4287542.24, 458235.144 42..."
2,"POLYGON ((458235.144 4289542.24, 458235.144 42..."
3,"POLYGON ((458235.144 4291542.24, 458235.144 42..."
4,"POLYGON ((458235.144 4293542.24, 458235.144 42..."


In [153]:
grid.to_file("output/portugal_grid.shp")
grid.to_file("output/portugal_grid.gpkg", layer="grid", driver="GPKG")

In [142]:
# add columns for coordinates of grid squares' centers
grid_gdf = grid.copy()
grid_gdf["centroid"] = grid.geometry.centroid
grid_gdf["centroid_x"] = grid.geometry.centroid.x
grid_gdf["centroid_y"] = grid.geometry.centroid.y
grid_gdf.head()

,geometry,centroid,centroid_x,centroid_y
0,"POLYGON ((458235.144 4285542.24, 458235.144 42...",POINT (457235.144 4286542.24),457235.143906,4.286542e+06
1,"POLYGON ((458235.144 4287542.24, 458235.144 42...",POINT (457235.144 4288542.24),457235.143906,4.288542e+06
2,"POLYGON ((458235.144 4289542.24, 458235.144 42...",POINT (457235.144 4290542.24),457235.143906,4.290542e+06
3,"POLYGON ((458235.144 4291542.24, 458235.144 42...",POINT (457235.144 4292542.24),457235.143906,4.292542e+06
4,"POLYGON ((458235.144 4293542.24, 458235.144 42...",POINT (457235.144 4294542.24),457235.143906,4.294542e+06


In [143]:
fire_data = fire_data.to_crs(coord_ref_sys)
